In [1]:
from scipy import signal
import numpy as np
from scipy.fftpack import fft, ifft
import pandas as pd

import matplotlib
matplotlib.use('nbagg')
from pylab import rcParams
rcParams['figure.figsize'] = 9, 4
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
import sys
import copy

from pyha import Hardware, simulate, sims_close, Complex, resize, hardware_sims_equal
from pathlib import Path
from data import load_iq

def imshow(im):
    from skimage.exposure import exposure
    p2, p98 = np.percentile(im, (2, 98))
    im = exposure.rescale_intensity(im, in_range=(p2, p98))

    
    plt.imshow(im, interpolation='nearest', aspect='auto', origin='lower')
    plt.tight_layout()
    plt.show()
    
    
def snr(pure, noisy):
    sig_pow = np.mean(np.abs(pure))
    error = np.array(pure) - np.array(noisy)
    err_pow = np.mean(np.abs(error))
    
    snr_db = 20*np.log10(sig_pow/err_pow)
    return snr_db


In [32]:
fft_points = 8
overlap = 2

In [33]:
orig_inp = np.array(list(range(64)))
print(orig_inp, orig_inp.shape)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63] (64,)


In [34]:
step = fft_points // overlap
chunks = []
for i in range(0, len(orig_inp) - fft_points, step):
    new = np.array(orig_inp[i: i+fft_points])
    chunks.append(new)
chunks = np.array(chunks)
print(chunks)

[[ 0  1  2  3  4  5  6  7]
 [ 4  5  6  7  8  9 10 11]
 [ 8  9 10 11 12 13 14 15]
 [12 13 14 15 16 17 18 19]
 [16 17 18 19 20 21 22 23]
 [20 21 22 23 24 25 26 27]
 [24 25 26 27 28 29 30 31]
 [28 29 30 31 32 33 34 35]
 [32 33 34 35 36 37 38 39]
 [36 37 38 39 40 41 42 43]
 [40 41 42 43 44 45 46 47]
 [44 45 46 47 48 49 50 51]
 [48 49 50 51 52 53 54 55]
 [52 53 54 55 56 57 58 59]]


In [35]:
chunks.shape

(14, 8)

In [36]:
chunks = np.resize(chunks, (len(chunks)//overlap, overlap, 8))
print(chunks)
print(chunks.shape)

[[[ 0  1  2  3  4  5  6  7]
  [ 4  5  6  7  8  9 10 11]]

 [[ 8  9 10 11 12 13 14 15]
  [12 13 14 15 16 17 18 19]]

 [[16 17 18 19 20 21 22 23]
  [20 21 22 23 24 25 26 27]]

 [[24 25 26 27 28 29 30 31]
  [28 29 30 31 32 33 34 35]]

 [[32 33 34 35 36 37 38 39]
  [36 37 38 39 40 41 42 43]]

 [[40 41 42 43 44 45 46 47]
  [44 45 46 47 48 49 50 51]]

 [[48 49 50 51 52 53 54 55]
  [52 53 54 55 56 57 58 59]]]
(7, 2, 8)


In [37]:
sums = np.mean(chunks, axis=1)
print(sums)
print(sums.shape)

[[ 2.  3.  4.  5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14. 15. 16. 17.]
 [18. 19. 20. 21. 22. 23. 24. 25.]
 [26. 27. 28. 29. 30. 31. 32. 33.]
 [34. 35. 36. 37. 38. 39. 40. 41.]
 [42. 43. 44. 45. 46. 47. 48. 49.]
 [50. 51. 52. 53. 54. 55. 56. 57.]]
(7, 8)


# Config

In [ ]:
fft_points = 1024 * 8
freq_axis_avg_decimation = 64
sims = ['MODEL', 'PYHA']

# Prepare input

In [ ]:
# orig_inp = load_iq('/home/gaspar/git/pyhacores/data/f2404_fs16.896_one_hop.iq')
orig_inp = load_iq('/home/gaspar/git/m037_tests/data/Phantom 3 & 4=-90_20180104123014261769/1515061814.9318_qdetector_fs=20000000.0_bw=20000000.0_fc=2420000000.0_d=0.raw')
# orig_inp = orig_inp.imag
# orig_inp = signal.decimate(orig_inp, 8)
# orig_inp *= 0.5

# n = np.random.uniform(-1, 1, len(orig_inp)) + np.random.uniform(-1, 1, len(orig_inp))*1j
# n *= 0.7
# orig_inp += n

# make sure input divides with fft_points
orig_inp = np.array(orig_inp[:int(len(orig_inp) // fft_points) * fft_points])

In [ ]:
# base = 0.5 / fft_points / 2
base = 1.0 / fft_points / 2

print(base, 0.0001)
overlap = 64
# taps = signal.firwin2(fft_points * overlap, [0, base, base, 1.0], [1, 1, 0, 0])
taps = signal.remez(fft_points * overlap, [0, base, base+0.000000001, 0.5], [1, 0], maxiter=20)

# taps /= taps.max()
w, h = signal.freqz(taps)

plt.plot(w, 20 * np.log10(abs(h)), 'b')
plt.ylabel('Amplitude [dB]', color='b')
plt.xlabel('Frequency [rad/sample]')
plt.show()

plt.plot(taps)
plt.show()
print(taps[0])

In [ ]:
f = np.fft.fft(taps)

In [ ]:
a = 20 * np.log10(abs(f))
a = np.fft.fftshift(a)
plt.plot(a)
plt.show()

In [ ]:
import scipy.signal
inp = [0.0 + 0.0j] * fft_points * overlap
inp[0] = 1.0 + 1.0j

fir = scipy.signal.lfilter(taps, [1.0], inp)


In [ ]:
plt.magnitude_spectrum(fir, window=matplotlib.mlab.window_none, scale='dB')

plt.ylabel('Amplitude [dB]')
plt.xlabel('Frequency')
plt.grid()
plt.show()

# Golden output

In [ ]:
_, _, spectro_out = signal.spectrogram(orig_inp, 1, nperseg=fft_points, return_onesided=False, detrend=False,
                               noverlap=fft_points * 0.5, window='hann')

# fftshift
spectro_out = np.roll(spectro_out, fft_points//2, axis=0)

# avg decimation
x = np.split(spectro_out, len(spectro_out) // freq_axis_avg_decimation)
golden_output = np.average(x, axis=1)

imshow(golden_output)

# MANUAL

In [ ]:
# divide to chunks of size 'nfft'
chunks = np.split(orig_inp, len(orig_inp) // fft_points)

# apply window
windowed = chunks * np.hanning(fft_points)

# take fft, this also fixes ordering
ffts = np.fft.fft(windowed).T

# take magnitude
mag = np.conjugate(ffts) * ffts
mag = mag.real

# # fftshift
spectro_out = np.roll(mag, fft_points//2, axis=0)

# avg decimation
x = np.split(spectro_out, len(spectro_out) // freq_axis_avg_decimation)
golden_output = np.average(x, axis=1)

imshow(golden_output)
# imshow(mag)

# MANUAL PRESUM

In [ ]:
# divide to chunks of size 'nfft'
chunks = np.array(np.split(orig_inp, len(orig_inp) // fft_points))

# apply window
# windowed = chunks * np.hanning(fft_points)
windowed = chunks * taps
# windowed = chunks

print(windowed.shape)
# presum

presum = []
for row in windowed:
#     print(row.shape)
    x = np.array(np.split(row, freq_axis_avg_decimation))
#     print(x.shape)
    golden_output = np.sum(x, axis=0)
#     print(golden_output.shape)
    presum.append(golden_output)
    # print(golden_output)
#     windowed = golden_output
presum = np.array(presum)
# take fft, this also fixes ordering
ffts = np.fft.fft(presum).T

# take magnitude
mag = np.conjugate(ffts) * ffts
mag = mag.real

# fftshift
spectro_out = np.roll(mag, fft_points//2// freq_axis_avg_decimation, axis=0)

# # avg decimation
# x = np.split(spectro_out, len(spectro_out) // freq_axis_avg_decimation)
# golden_output = np.average(x, axis=1)

imshow(spectro_out)

In [ ]:
# divide to chunks of size 'nfft'
chunks = orig_inp[10*fft_points: 11*fft_points]

# apply window
windowed = chunks * np.hanning(fft_points)
# windowed = chunks
# print(windowed.shape)

# take fft, this also fixes ordering
ffts = np.fft.fft(windowed)

# take magnitude
mag = np.conjugate(ffts) * ffts
mag = mag.real

# # fftshift
spectro_out = np.roll(mag, fft_points//2, axis=0)

# avg decimation
# golden_output = spectro_out
x = np.split(spectro_out, len(spectro_out) // freq_axis_avg_decimation)
golden_output = np.average(x, axis=1)

plt.plot(golden_output)
plt.show()
# imshow(mag)

In [ ]:
0.5 / 256

In [ ]:
# divide to chunks of size 'nfft'
chunks = orig_inp[10*fft_points: 11*fft_points]

# apply window
# windowed = chunks * np.hanning(fft_points)
plt.plot(taps)
plt.show()
windowed = chunks * taps
# windowed = np.array(chunks)


# presum
# x = np.array(np.split(windowed, 2))
# golden_output = np.average(x, axis=0)
# windowed = golden_output

windowed = windowed[:fft_points//2] + windowed[fft_points//2:]

# take fft, this also fixes ordering
ffts = np.fft.fft(windowed)

# take magnitude
mag = np.conjugate(ffts) * ffts
mag = mag.real

# fftshift
spectro_out = np.roll(mag, fft_points//2//2, axis=0)

# # avg decimation
# x = np.split(spectro_out, len(spectro_out) // freq_axis_avg_decimation)
# golden_output = np.average(x, axis=1)

plt.plot(spectro_out)
plt.show()

# imshow(mag)

In [ ]:
l = [0.0] * 10 + [1.0] * 10 + [0.0] * 10
plt.plot(l)
plt.show()

iff = np.fft.ifft(l)

plt.plot(abs(iff))
plt.show()

In [ ]:
MAKSA ARVE!!!

# Packager

In [ ]:
dut = Packager(fft_points)
sims = simulate(dut, orig_inp, output_callback=DataWithIndex._pyha_unpack, simulations=sims)
assert sims_close(sims)

In [ ]:
snrs = snr(sims['MODEL'], sims['PYHA'])
print(f'Block AVG SNR: {snrs}')

# Apply windowing

In [ ]:
dut = Windower(fft_points)

In [ ]:
inp = np.array(sims['PYHA'])
sims = simulate(dut, inp, simulations=sims, output_callback=DataWithIndex._pyha_unpack, input_callback=DataWithIndex._pyha_pack)
assert hardware_sims_equal(sims)

In [ ]:
plt.plot(np.hstack(sims['MODEL']).real, label='MODEL')
plt.plot(np.hstack(sims['PYHA']).real, label='HARDWARE')
plt.legend()
plt.show()

In [ ]:
snrs = snr(sims['MODEL'], sims['PYHA'])
print(f'Block AVG SNR: {snrs}')

# FFT

In [ ]:
inp = np.array(sims['PYHA']) # input is the output of last block
dut = R2SDF(fft_points)
sims = simulate(dut, inp, simulations=sims, output_callback=DataWithIndex._pyha_unpack, input_callback=DataWithIndex._pyha_pack)
assert hardware_sims_equal(sims)

In [ ]:
plt.plot(np.hstack(sims['MODEL']).real, label='MODEL')
plt.plot(np.hstack(sims['PYHA']).real, label='HARDWARE')
plt.legend()
plt.show()

In [ ]:
snrs = snr(sims['MODEL'], sims['PYHA'])
print(f'Block AVG SNR: {snrs}')

# Bit-reversal and fftshift

In [ ]:
inp = np.array(sims['PYHA']) # input is the output of last block
dut = BitReversal(fft_points)
sims = simulate(dut, inp, simulations=sims, output_callback=DataWithIndex._pyha_unpack, input_callback=DataWithIndex._pyha_pack)
assert hardware_sims_equal(sims)


In [ ]:
plt.plot(np.hstack(sims['MODEL']).real, label='MODEL')
plt.plot(np.hstack(sims['PYHA']).real, label='HARDWARE')
plt.legend()
plt.show()

In [ ]:
snrs = snr(sims['MODEL'], sims['PYHA'])
print(f'Block AVG SNR: {snrs}')

# Magnitude

In [ ]:
inp = np.array(sims['PYHA']) # input is the output of last block
dut = ConjMult()
sims = simulate(dut, inp, simulations=sims, output_callback=DataWithIndex._pyha_unpack, input_callback=DataWithIndex._pyha_pack)
assert hardware_sims_equal(sims)

In [ ]:
plt.plot(np.hstack(sims['MODEL']).real, label='MODEL')
plt.plot(np.hstack(sims['PYHA']).real, label='HARDWARE')
plt.legend()
plt.show()

In [ ]:
snrs = snr(sims['MODEL'], sims['PYHA'])
print(f'Block AVG SNR: {snrs}')

# Decimation AVG

In [ ]:
inp = np.array(sims['PYHA']) # input is the output of last block
dut = AvgDecimate(freq_axis_avg_decimation)
sims = simulate(dut, inp, simulations=sims, output_callback=DataWithIndex._pyha_unpack, input_callback=DataWithIndex._pyha_pack)
assert hardware_sims_equal(sims)

In [ ]:
plt.plot(np.hstack(sims['MODEL']).real, label='MODEL')
plt.plot(np.hstack(sims['PYHA']).real, label='HARDWARE')
plt.legend()
plt.show()

In [ ]:
snrs = snr(sims['MODEL'], sims['PYHA'])
print(f'Block AVG SNR: {snrs}')

# Merged object

In [ ]:
inp = orig_inp
dut = Spectrogram(fft_points, decimate_by=freq_axis_avg_decimation)
full_sims = simulate(dut, inp, simulations=sims, output_callback=DataWithIndex._pyha_unpack)
assert hardware_sims_equal(sims)

In [ ]:
x = np.array(full_sims['PYHA']).T
imshow(x)

# Final evaluation

In [ ]:
x = np.array(sims['PYHA']).T
imshow(x)

In [ ]:
imshow(golden_output)

In [ ]:
flat_spectro = golden_output.flatten()
flat_spectro /= golden_output.max()

flat_pyha = x.flatten()
flat_pyha /= flat_pyha.max()

plt.plot(flat_spectro, label='MODEL_PYHA')
plt.plot(flat_pyha, label='HARDWARE')
plt.legend()
plt.show()

plt.plot(flat_pyha - flat_spectro, label='MODEL_PYHA')
plt.legend()
plt.show()

snrs = snr(flat_spectro, flat_pyha)
print(f'Block AVG SNR: {snrs}')
